In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'


# dx = 250 m
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
#JOB ARRAY SETUP
job_array=True
if job_array==True:

    num_jobs=60 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***
    total_elements=len(data['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=46
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    print(f'start_job = {start_job}, end_job = {end_job}')

In [ ]:
#Indexing Array with JobArray
data=data.isel(time=slice(start_job,end_job))
parcel=parcel.isel(time=slice(start_job,end_job))
#(for 150_000_000 parcels use 500-1000 jobs)

In [ ]:
#constants
Cp=1004 #Jkg-1K-1
Cv=717 #Jkg-1K-1
Rd=Cp-Cv #Jkg-1K-1
eps=0.608

Lx=(data['xf'][-1].item()-data['xf'][0].item())*1000 #x length (m)
Ly=(data['yf'][-1].item()-data['yf'][0].item())*1000 #y length (m)
Np=len(parcel['xh']) #number of lagrangian parcles
dt=(data['time'][1]-data['time'][0]).item()/1e9 #sec
dx=(data['xf'][1].item()-data['xf'][0].item())*1e3 #meters
dy=(data['yf'][1].item()-data['yf'][0].item())*1e3 #meters
xs=data['xf'].values*1000
ys=data['yf'].values*1000
zs=data['zf'].values*1000

def zf(z):
    k=z #z is the # level of z
    out=data['zf'].values[k]*1000
    
    return out
# def rho(x,y,z,t):
#     p=data['prs'].isel(xh=x,yh=y,zh=z,time=t).item()
#     p0=101325 #Pa
#     theta=data['th'].isel(xh=x,yh=y,zh=z,time=t).item()
#     T=theta*(p/p0)**(Rd/Cp)
#     qv=data['qv'].isel(xh=x,yh=y,zh=z,time=t).item()
#     # Tv=T*(1+eps*qv)
#     Tv=T*(eps+qv)/(eps*(1+qv))
#     rho = p/(Rd*Tv)
#     out=rho
#     return out

def rho(x,y,z,rho_data_t):
    out=rho_data_t[z,y,x]
    return out
def m(t):
    rho_data_t=data['rho'].isel(time=t).data
    
    m=0
    #triple sum
    for k in range(len(data['zh'])):
        dz=(zf(k+1)-zf(k))
        for j in range(len(data['yh'])):
            for i in range(len(data['xh'])):
                rho_out=rho(i,j,k,rho_data_t)
                m+=rho_out*dz
                
    #triple sum
    out=m*dx*dy/Np
    return out


In [ ]:

#Calculate Mass Constant
# calculate='single_time'
# calculate=True
calculate=False

if calculate==True:
    Nt=len(data['time'])
    m_arr=np.zeros((Nt))
    for t in np.arange(Nt):
        if np.mod(t,25)==0: print(t)
        m_arr[t]=m(t)
    np.save(f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
elif calculate=='single_time':
    Nt=len(data['time'])
    m_arr=np.zeros((Nt))

    t=len(data['time'])//2 #Pick some middle time
    m_300=m(t)
    for t in np.arange(Nt):
        m_arr[t]=m_300 #UNCOMMENT FOR FULL CALCULATION
    np.save(f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
else:
    dir3=dir+f'Project_Algorithms/Entrainment/'
    m_arr = np.load(f'Mass_Array_{res}_{t_res}_{Np_str}.npy')

# # TESTING
# lst=[]
# for t in np.arange(133):
#     lst.append(m_arr[t])

# plt.plot(lst)
# (np.max(lst)-np.min(lst))*100/np.mean(lst)

In [ ]:
#READING BACK IN
PROCESSING=False
PROCESSING=True

print('loading vars')
if PROCESSING==False:
    dir3=dir+f'Project_Algorithms/Entrainment/3D_VMF_profiles_{res}_{t_res}_{Np_str}.h5'
if PROCESSING==True:
    dir3=dir+f'Project_Algorithms/Entrainment/3D_VMF_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
with h5py.File(dir3, "r") as h5f:
    profile_array_VMF_g = h5f["profile_array_VMF_g"][start_job:end_job]
    profile_array_VMF_c = h5f["profile_array_VMF_c"][start_job:end_job]


def apply_constant(profile_array,apply):
    if apply==True:
        Nt=profile_array.shape[0]
        Nz=profile_array.shape[1]
    
        profile_array/=(dx*dy)
        for t in np.arange(Nt):
            profile_array[t]*=m_arr[t+index_adjust]
        for z in np.arange(Nz):
            dz=zf(z+1)-zf(z)
            profile_array[:,z]/=dz
    return profile_array

#APPLY CONSTANTS TO VMF VALUE
##################################################
profile_array_VMF_g=apply_constant(profile_array_VMF_g,apply=True)
profile_array_VMF_c=apply_constant(profile_array_VMF_c,apply=True)
##################################################

print('done')

In [ ]:
def call_variables(): #***
    variable='winterp'; w_data=data[variable].data
    variable='qv'; qv_data=data[variable].data # get qc data
    variable='qc'; qc_data=data[variable].data # get qc data
    variable='qi'; qi_data=data[variable].data # get qc data
    qc_plus_qi=qc_data+qi_data
    return w_data,qv_data,qc_data,qi_data,qc_plus_qi

print('calling variables')
[w_data,qv_data,qc_data,qi_data,qc_plus_qi]=call_variables()
print('done')

In [ ]:
##########################################################################################
#Functions
# Full Profile function makes profile together for all timesteps. AveragedProfiles funciton takes the final mean of the combined profile.

In [ ]:
# Thresholds
w_thresh1 = 0.1
w_thresh2 = 0.5
qcqi_thresh = 1e-6

def final_profile(vars_list, type):
    zhs = data['zh'].values
    profiles = {}  # Store profiles for all variables

    # Initialize profiles for each variable
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs

    # Threshold mask
    if type == "general":
        where_updraft = (w_data >= w_thresh1) & (qc_plus_qi < qcqi_thresh)
    elif type == "cloudy":
        where_updraft = (w_data >= w_thresh2) & (qc_plus_qi >= qcqi_thresh)
        
    t_ind, z_ind, y_ind, x_ind = np.where(where_updraft)

    # Variable selection dictionary
    var_data = {
        'VMF_g': profile_array_VMF_g, 'VMF_c': profile_array_VMF_c, 
    }
    
    # Iterate over each variable in vars_list and bin the data
    for var in vars_list:
        masked_data = var_data[var][where_updraft]
        np.add.at(profiles[var][:, 0], z_ind, masked_data)
        np.add.at(profiles[var][:, 1], z_ind, 1)

    return profiles

In [ ]:
##########################################################################################
#Running

In [ ]:
dir2=dir+'Project_Algorithms/Domain_Profiles/'

print(f"job_id = {job_id}")
for type in ["general","cloudy"]:
    print(f"working on {type} type\n")
    
    vars_list = ['VMF_g', 'VMF_c']
    profiles = final_profile(vars_list, type)
    
    #Saving eulerian_profiles
    import h5py
    
    if type == "general":
        output_file = dir2+f'job_out/general_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5' 
    elif type == "cloudy":
        output_file = dir2+f'job_out/cloudy_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5'
    
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'profile_{var}', data=profile_var, compression="gzip")

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB_ARRAYS IS RUNNING
# recombine=True

In [ ]:
if recombine==True:
    dir2=dir+'Project_Algorithms/Domain_Profiles/'
    
    types=['general','cloudy']
    for type in types:
        #MAKING OUTPUT FILE PATH
        if type == "general":
            output_file =dir2+f'job_out/general_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}.h5'
        elif type == "cloudy":
            output_file = dir2+f'job_out/cloudy_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}.h5'
        
        #MAKING PROFILES DICTIONARY
        zhs = data['zh'].values
        profiles = {}  # Store profiles for all variables
        vars_list = ['VMF_g', 'VMF_c']
        for var in vars_list:
            profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
            profiles[var][:, 2] = zhs 
        
        num_jobs=60
        for job_id in np.arange(1,num_jobs+1):
            if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
    
            #CALLING IN DATA
            if type == "general":
                input_file = dir2+f'job_out/general_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5' 
            elif type == "cloudy":
                input_file = dir2+f'job_out/cloudy_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5' 
    
            #COMPILING PROFILES
            with h5py.File(input_file, 'r') as f:
                for var in vars_list:  
                    profiles[var][:,0:1+1]+=f[f'profile_{var}'][:,0:1+1]
        
        #SAVING INTO FINAL FORM
        with h5py.File(output_file, 'w') as f:
            for var in profiles:
                profile_var = profiles[var]
                f.create_dataset(f'profile_{var}', data=profile_var, compression="gzip")

In [ ]:
##########################################################################################
#Plotting
plotting=False #KEEP FALSE WHEN JOB_ARRAYS IS RUNNING
# plotting=True

In [ ]:
def averaged_profiles(profile):
    out_var = profile[(profile[:, 1] != 0)]  # gets rid of rows that have no data
    out_var = np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T  # divides the data column by the counter column
    return out_var

In [ ]:
if plotting==True:
    for type in ["general","cloudy"]:
        print(f'currently on type {type}')
        dir2=dir+'Project_Algorithms/Domain_Profiles/'
        if type == "general":
            input_file =dir2+f'job_out/general_eulerian_VMF_profiles_{res}_{Np_str}_5min.h5'#_{46}.h5'
        elif type == "cloudy":
            input_file =dir2+f'job_out/cloudy_eulerian_VMF_profiles_{res}_{Np_str}_5min.h5'#_{46}.h5'
        
        with h5py.File(input_file, 'r') as f:
            if type == "general":
                VMF = np.array(f['profile_VMF_g'])
            elif type == "cloudy":
                VMF = np.array(f['profile_VMF_c'])
        
        
        #Uses Averaged_Profiles Function
        vars = ['VMF'] 
        
        for var in vars:
            globals()[f"out_{var}"] = averaged_profiles(globals()[f"{var}"])
        
        if type=='general':
            color='black'
        elif type=='cloudy':
            color='blue'
    
        plt.plot(out_VMF[:,0],out_VMF[:,1],color=color,label=type)
        plt.axvline(0,color='k')
        ax = plt.gca()
        apply_scientific_notation([ax])
        
        plt.legend()

In [ ]:
# #DOUBLE CHECKING
# w=data['winterp'].data
# qc=data['qc'].data
# qi=data['qi'].data
# qcqi=qc+qi
# mask = (w >= 0.5) & (qcqi >= 1e-6)
# profile_array_VMF_c[~mask] = np.nan
# out=np.nanmean(profile_array_VMF_c,axis=(0,2,3))
# plt.plot(out,data['zh'])

In [ ]:
if plotting==True:
    def average_difference(array1, array2):
        out_var_one=averaged_profiles(array1)
        out_var_two=averaged_profiles(array2)
    
        #masking out non matches
        second_col_one = out_var_one[:, 1]
        second_col_two = out_var_two[:, 1]
        mask_one = np.isin(second_col_one, second_col_two)
        mask_two = np.isin(second_col_two, second_col_one)
        
        out_var_one = out_var_one.copy()[mask_one]
        out_var_two = out_var_two.copy()[mask_two]
        
        diff=(out_var_one[:,0]-out_var_two[:,0])
        zs=out_var_one[:,1]
    
        out_profile=np.zeros((len(diff),2))
    
        out_profile[:,0]=diff;out_profile[:,1]=zs;
        return out_profile

In [ ]:
if plotting==True:
    #Plotting Differences
    def averaged_profiles(profile):
        out_var = profile[(profile[:, 1] != 0)]  # gets rid of rows that have no data
        # out_var = np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T  # divides the data column by the counter column
        Nt=len(data['time']);Ny=len(data['yh']);Nx=len(data['xh'])
        out_var[:,0]/=(Ny*Nx*Nt)
        out_var = np.array([out_var[:, 0], out_var[:, 2]]).T  # divides the data column by the counter column
        return out_var
    label=1
    
        
    def get_data(type):
        dir2=dir+'Project_Algorithms/Domain_Profiles/'
        if type == "general":
            input_file = dir2+f'job_out/general_eulerian_VMF_profiles_{res}_{Np_str}_5min.h5' 
        elif type == "cloudy":
            input_file = dir2+f'job_out/cloudy_eulerian_VMF_profiles_{res}_{Np_str}_5min.h5' 
    
        with h5py.File(input_file, 'r') as f:
            if type == "general":
                VMF = np.array(f['profile_VMF_g'])
            elif type == "cloudy":
                VMF = np.array(f['profile_VMF_c'])
    
        return VMF
    
        
    def diff_plotting():
        #setting up figure
        fig, axis = plt.subplots(1, 1, figsize=(8, 6))
        # fig.suptitle("")
        # ax1= axs.flatten()
    
        #the plotting
        vars = ['VMF'] 
        xlabels = ['VMF'] 
        for xlabel,var in zip(xlabels,vars):
    
            #get profile
            types=['cloudy','general']
            out_var_one=get_data(types[0])
            out_var_two=get_data(types[1])
    
            #finding where cloudy updraft count goes below 10
            cutofflow=np.where(out_var_one[:,1]>10)[0][0]
            cutoffhigh=np.where(out_var_one[:,1]>10)[0][-1]
           
            #averaging
            out_var_diff=average_difference(out_var_one,out_var_two)
            
            axis.plot(out_var_diff[:,0],out_var_diff[:,-1],color='k')
            axis.axvline(0,color='k',linestyle='dashed')
            # axis.set_ylim(bottom=0,top=20)
            ## axis.set_ylim(bottom=data['zh'][cutofflow],top=data['zh'][cutoffhigh])
            #labeling
            axis.set_ylabel('z (km)');axis.set_xlabel(xlabel);
            axis.grid(True)
            # axis.legend(fontsize='small') #only adds legend at final variable
    
            if axis==axis:
                apply_scientific_notation([axis])
    
            # axis.set_yticks(list(axis.get_yticks()) + [data['zh'][cutofflow]]) #TESTING
        plt.tight_layout()
    
    diff_plotting()